In [61]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from xgboost import XGBClassifier
import numpy as np
import tkinter as tk
from tkinter import messagebox

# 加载数据集
df = pd.read_csv('Dataset of Diabetes .csv')

# 删除 ID 和 No_Pation 列
df = df.drop(['ID', 'No_Pation'], axis=1)

# 对 Gender 和 CLASS 进行编码
label_encoders = {}
for column in ['Gender', 'CLASS']:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# 划分特征和目标变量
X = df.drop('CLASS', axis=1)
y = df['CLASS']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 构建并训练神经网络模型
model_nn = Sequential()
model_nn.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_nn.add(Dense(32, activation='relu'))
model_nn.add(Dense(len(np.unique(y)), activation='softmax'))

model_nn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_nn.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# 构建并训练 XGBoost 模型
model_xgb = XGBClassifier()
model_xgb.fit(X_train, y_train)


D:\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [73]:
def predict(entries):
    try:
        # 单独处理 Gender 输入（保持字符串）
        gender = entries[1].strip().upper()
        if gender not in ['F', 'M']:
            messagebox.showerror("INPUT ERROR", "Gender must be F or M")
            return

        # 其他字段转换为 float
        input_data = [
            gender,  # 保持字符串用于编码
            float(entries[2]),
            float(entries[3]),
            float(entries[4]),
            float(entries[5]),
            float(entries[6]),
            float(entries[7]),
            float(entries[8]),
            float(entries[9]),
            float(entries[10]),
            float(entries[11])
        ]

        # 对 Gender 进行编码
        encoded_gender = label_encoders['Gender'].transform([input_data[0]])[0]
        input_data[0] = encoded_gender

        input_data = np.array(input_data).reshape(1, -1)

        # 使用两个模型进行预测
        pred_nn = model_nn.predict(input_data).argmax(axis=1)[0]
        pred_xgb = model_xgb.predict(input_data)[0]

        # 平均集成预测结果
        final_prediction = (pred_nn + pred_xgb) // 2

        # 将编码结果转换回原始类别
        result = label_encoders['CLASS'].inverse_transform([final_prediction])[0]
        #messagebox.showinfo("Forest result", f"Result: {result}")
        return result
        
    except Exception as e:
        messagebox.showerror("error", {e})



In [155]:
data = ["LI","M",23,33,2,4,2,3,4,2,2,3,2,4]
final_result = predict(data)
suggestion = " "
if final_result == "Y" :
    suggestion = "According to the data, you have a higher risk of developing diabetes. Please consult a regular hospital for medical examination as soon as possible"
if final_result == "P" :
    suggestion = "According to the data,Your current body is more likely to develop diabetes, please improve the current diet and lifestyle habits."
if final_result == "N":
    suggestion == "According to the data, Your standard of living is healthy, please keep it up"

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [161]:
import gradio as gr
import pandas as pd
 
def greet(name,gender,age,urea,cr,hbalc,chol,tg,hdl,ldl,vldl,bmi):
    data = [name,gender,age,urea,cr,hbalc,chol,tg,hdl,ldl,vldl,bmi]
    final_result = predict(data)
    
    suggestion = " "
    if final_result == "Y" :
        suggestion = "According to the data, you have a higher risk of developing diabetes. Please consult a regular hospital for medical examination as soon as possible"
    if final_result == "P" :
        suggestion = "According to the data,Your current body is more likely to develop diabetes, please improve the current diet and lifestyle habits."
    if final_result == "N":
        suggestion == "According to the data, Your standard of living is healthy, please keep it up"
    
    return f"{final_result}: " + f"{suggestion}"
 
custom_css = """
/* 外层容器居中 */
.gradio-container {
    max-width: 800px;  /* 控制页面宽度 */
    margin: 0 auto !important;  /* 水平居中 */
    padding: 20px !important;
}

/* 标题居中 */
#title {
    text-align: center;
    font-size: 24px;
    margin-bottom: 30px !important;
}
"""

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("""<h1 id="title">Diabetes Health Survey</h1>""")
    
    with gr.Row():
        name = gr.Textbox(label="Name")
        gender = gr.Radio(choices=["M", "F"], label="Gender")
        age = gr.Number(label="Age", minimum=0, maximum=100, step=1)
    
    with gr.Row():
        urea = gr.Slider(label="Urea (Normal Range:2.5-7.1)", minimum=1, maximum=10)
        cr = gr.Slider(label="Cr (Normal Range:44-113)", minimum=1, maximum=150)
        hbalc = gr.Slider(label="HbA1c (Normal Range:4-6)", minimum=1, maximum=10)
        chol = gr.Slider(label = "Chol(Normal Range:2.8-5.7)",minimum = 1,maximum = 10)
        tg = gr.Slider(label = "TG(Normal Range:0.45-1.7)",minimum = 0.1,maximum = 2)
        hdl = gr.Slider(label = "HDL(Normal Range:0.9-2.2)",minimum = 0.1,maximum = 5)
        ldl = gr.Slider(label = "LDL(Normal Range:2.07-3.37)",minimum = 0.1,maximum = 5)
        vldl = gr.Slider(label = "VLDL(Normal Range:0.1-0.8)",minimum = 0.1,maximum = 2)
        bmi = gr.Slider(label = "BMI(Normal Range:18.5-23)",minimum = 1,maximum = 50)
        
    with gr.Row():
        output = gr.Textbox(label="Prediction Result")

        
    with gr.Row():
        submit = gr.Button("Submit", variant="primary")
        clear = gr.Button("Clear", variant = "secondary")

#按钮监听器
    submit.click(
        fn=greet,
        inputs=[name, gender, age, urea, cr, hbalc, chol, tg, hdl, ldl, vldl, bmi],
        outputs=output
    )

    clear.click(
            fn=lambda: [
                gr.update(value=""),  # Name
                gr.update(value=None),  # Gender
                gr.update(value=0),  # Age
                gr.update(value=1),  # Urea
                gr.update(value=1),  # Cr
                gr.update(value=1),  # HbA1c
                gr.update(value=1),  # Chol
                gr.update(value=0.1),  # TG
                gr.update(value=0.1),  # HDL
                gr.update(value=0.1),  # LDL
                gr.update(value=0.1),  # VLDL
                gr.update(value=1),  # BMI
                gr.update(value="")  # Output
            ],
            inputs=None,
            outputs=[name, gender, age, urea, cr, hbalc, chol, tg, hdl, ldl, vldl, bmi, output]
        )


demo.launch(inbrowser=True)



# iface = gr.Interface(
#     allow_flagging = "never",
#     title = "Diabete Health Survey",
#     fn=greet,
#     inputs=[
#         gr.Textbox(label = "Name"),
#         gr.Radio(choices = ["M","F"],label = "Gender"),
#         gr.Number(
#             label = "age",
#             minimum = 0,
#             maximum = 100,
#             step = 1
#         ),
#         gr.Slider(label = "Urea(Normal Range:2.5-7.1)", minimum = 1,maximum = 10),
#         gr.Slider(label = "Cr(Normal Range:44-113)",minimum = 1,maximum = 150),
#         gr.Slider(label = "HbAlc(Normal Range:4-6)",minimum = 1,maximum = 10),
#         gr.Slider(label = "Chol(Normal Range:2.8-5.7)",minimum = 1,maximum = 10),
#         gr.Slider(label = "TG(Normal Range:0.45-1.7)",minimum = 0.1,maximum = 2),
#         gr.Slider(label = "HDL(Normal Range:0.9-2.2)",minimum = 0.1,maximum = 5),
#         gr.Slider(label = "LDL(Normal Range:2.07-3.37)",minimum = 0.1,maximum = 5),
#         gr.Slider(label = "VLDL(Normal Range:0.1-0.8)",minimum = 0.1,maximum = 2),
#         gr.Slider(label = "BMI(Normal Range:18.5-23)",minimum = 1,maximum = 50),
        
        
#     ],
#     outputs=gr.Markdown(label = "Prediction result")
    
# )
 
# # 调用iface对象的launch方法，启动Web服务器并打开Web界面。
# # 用户可以在该界面上输入名字，点击提交后看到greet函数的输出结果。
# iface.launch(inbrowser = True)

* Running on local URL:  http://127.0.0.1:7913

To create a public link, set `share=True` in `launch()`.
